In [10]:

# For local testing
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow warnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
from tensorflow.keras.utils import image_dataset_from_directory
# Metrics、Plot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.random.set_seed(42)
np.random.seed(42)

In [11]:
data_folder = pathlib.Path('./dataset/')

### 分割 train、validation、test

In [12]:
# 載入訓練和驗證資料集（從 train 資料夾中分割 20% 作為驗證集）
train_dataset = image_dataset_from_directory(
    data_folder / "train",
    color_mode='grayscale',
    image_size=(224, 224),  # 調整為 224x224 以適配 VGG16
    batch_size=32,
    shuffle=True,
    label_mode='categorical',
    validation_split=0.2,
    subset="training",
    seed=42
)

val_dataset = image_dataset_from_directory(
    data_folder / "train",
    color_mode='grayscale',
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    label_mode='categorical',
    validation_split=0.2,
    subset="validation",
    seed=42
)

test_dataset = image_dataset_from_directory(
    data_folder / "test",
    color_mode='grayscale',
    image_size=(224, 224),
    batch_size=32,
    shuffle=False,
    label_mode='categorical'
)

class_names = train_dataset.class_names
print("Class names:", class_names)

# 定義函數將灰階圖像轉換為 RGB（複製單通道到三通道）
def grayscale_to_rgb(image, label):
    image = tf.image.grayscale_to_rgb(image)  # 將灰階圖像轉為 RGB
    return image, label

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.
Found 7178 files belonging to 7 classes.
Class names: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


## 正規化、加入緩存載入

In [13]:
# 應用轉換
train_dataset = train_dataset.map(grayscale_to_rgb)
val_dataset = val_dataset.map(grayscale_to_rgb)
test_dataset = test_dataset.map(grayscale_to_rgb)

# 正規化圖像像素值到 [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# 資料增強
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomTranslation(0.1, 0.1)
])
train_dataset_augmentation = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

# 優化資料載入
train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
train_dataset_augmentation = train_dataset_augmentation.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### Model 1 CNN basic 4 layer

In [14]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# 載入 VGG16 模型（不包括頂層全連接層，並使用預訓練權重）
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 凍結 VGG16 的預訓練層
base_model.trainable = False

# 構建模型
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)  # 使用 VGG16 提取特徵
x = layers.GlobalAveragePooling2D()(x)  # 全局平均池化，減少參數量
x = layers.Dense(256, activation='relu')(x)  # 全連接層
x = layers.Dropout(0.5)(x)  # 防止過擬合
outputs = layers.Dense(len(class_names), activation='softmax')(x)  # 7 個類別

# 創建模型
model_vgg16 = models.Model(inputs, outputs)

# 編譯模型（使用較小的學習率以適應遷移學習）
model_vgg16.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # 較小的學習率
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 顯示模型結構
model_vgg16.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,815 (56.64 MB)

 Trainable params: 133,127 (520.03 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

#### Training

In [ ]:
# 設置回調
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="./models/best_vgg16_model.keras",
        save_best_only=True,
        monitor="val_loss"
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]

# 訓練模型
history_vgg16 = model_vgg16.fit(
    train_dataset_augmentation,
    epochs=50,
    validation_data=val_dataset,
    callbacks=callbacks
)

Epoch 1/50
147/718 ━━━━━━━━━━━━━━━━━━━━ 4:19 455ms/step - accuracy: 0.2091 - loss: 2.0148

#### Plotting

In [ ]:
def plot_training_history(history, model):
    # 最終評估：在測試集上計算準確率和損失
    test_loss, test_acc = model.evaluate(test_dataset)
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_acc}")

    # 混淆矩陣與分類報告（使用 test_dataset）
    # 獲取真實標籤
    y_true = np.concatenate([y for x, y in test_dataset], axis=0)
    y_true = np.argmax(y_true, axis=1)

    # 預測標籤
    y_pred = model.predict(test_dataset)
    y_pred = np.argmax(y_pred, axis=1)

    # 檢查形狀是否一致
    if len(y_true) != len(y_pred):
        raise ValueError(f"Mismatch between y_true ({len(y_true)}) and y_pred ({len(y_pred)}) lengths!")

    # 打印混淆矩陣
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix (Test Set):\n", cm)

    # 繪製混淆矩陣熱圖
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix (Test Set)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # 打印分類報告
    print("Classification Report (Test Set):\n", classification_report(y_true, y_pred, target_names=class_names))

    # 繪製訓練曲線（不變）
    plt.figure(figsize=(10, 5))

    # 損失曲線
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss', color='blue')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
    plt.title('Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend()

    # 準確率曲線
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
    plt.title('Accuracy Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.legend()

    # 調整佈局並顯示
    plt.tight_layout()
    plt.show()

# 繪製訓練歷史
plot_training_history(history_vgg16, model_vgg16)